<a href="https://colab.research.google.com/github/usugunawan15/DTI-Telkom/blob/master/Tugas_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>